# Tarea 1 - Manuel Figueroa y Claudio Galaz

Primero se importan las librerías y los datos.

Los datos son los siguientes:
>**Prostate data info**

>Predictors (columns 1--8)

>outcome (column 9)

>train/test indicator (column 10)

>This last column indicates which 67 observations were used as the 
"training set" and which 30 as the test set.

>The goal is to predict the log of PSA (prostate specific antigen) (**lpsa**) from a number of measurements including log cancer volume (**lcavol**), log prostate weight **lweight**, **age**, log of benign prostatic hyperplasia amount **lbph**, seminal vesicle invasion **svi**, log of capsular penetration **lcp**, Gleason score gleason, and percent of Gleason scores 4 or 5 **pgg45**. 


In [12]:
import pandas as pd
import numpy as np

url = 'http://statweb.stanford.edu/~tibs/ElemStatLearn/datasets/prostate.data'
df = pd.read_csv(url, sep ='\t', header = 0)

Luego se elimina la primera columna de los datos, que contiene el número de fila.

In [13]:
df = df.drop('Unnamed: 0', axis=1)

Y después se crean arreglos con los datos de training/test set, y se elimina esa columna de los datos

In [14]:
istrain_str = df['train']
istrain = np.asarray([True if s == 'T' else False for s in istrain_str])
istest = np.logical_not(istrain)

df = df.drop('train', axis=1)

Finalmente quedan dos arreglos booleanos: **istrain** que es *True* para los datos que forman parte del training set e **istest** que es la operación lógica *NOT* sobre el primer set, marcando con *True* los elementos del test set.

Las lineas de a continuación muestran información relevante sobre los datos extraídos. Estos son la cantidad de datos (count), el promedio (mean), desviación estándar (std), dato máximo (max), dato mínimo (min), y los percentiles 25, 50 y 75.

In [15]:
#from IPython.display import clear_output
df.shape
df.info()
df.describe()
#clear_output()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 97 entries, 0 to 96
Data columns (total 9 columns):
lcavol     97 non-null float64
lweight    97 non-null float64
age        97 non-null int64
lbph       97 non-null float64
svi        97 non-null int64
lcp        97 non-null float64
gleason    97 non-null int64
pgg45      97 non-null int64
lpsa       97 non-null float64
dtypes: float64(5), int64(4)
memory usage: 7.6 KB


,lcavol,lweight,age,lbph,svi,lcp,gleason,pgg45,lpsa
count,97.000000,97.000000,97.000000,97.000000,97.000000,97.000000,97.000000,97.000000,97.000000
mean,1.350010,3.628943,63.865979,0.100356,0.216495,-0.179366,6.752577,24.381443,2.478387
std,1.178625,0.428411,7.445117,1.450807,0.413995,1.398250,0.722134,28.204035,1.154329
min,-1.347074,2.374906,41.000000,-1.386294,0.000000,-1.386294,6.000000,0.000000,-0.430783
25%,0.512824,3.375880,60.000000,-1.386294,0.000000,-1.386294,6.000000,0.000000,1.731656
50%,1.446919,3.623007,65.000000,0.300105,0.000000,-0.798508,7.000000,15.000000,2.591516
75%,2.127041,3.876396,68.000000,1.558145,0.000000,1.178655,7.000000,40.000000,3.056357
max,3.821004,4.780383,79.000000,2.326302,1.000000,2.904165,9.000000,100.000000,5.582932


De los datos se puede observar que la característica *svi* es binaria, *gleason* es entera con valores entre 6 y 9, y *pgg45* es una variable porcentual y entera.

Antes de entrar a trabajar con los datos es importante **normalizarlos**. Esto es necesario puesto que, como cada dimension tiene su propia escala de datos, se hace difícil comparar entre dimensiones. 

Por ejemplo, *age* varía entre 41 y 79 y *lweight* se mueve entre 2.374906 y 4.780383, por lo que para poder trabajar con estos datos en conjunto es importante llevarlos a una misma escala.

Una vez normalizada, la columna de la característica *lpsa* debe quedar como estaba antes (sin normalizar) debido a que es el objetivo a predecir.

In [16]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
df_scaled['lpsa'] = df['lpsa']
df_scaled.describe()

,lcavol,lweight,age,lbph,svi,lcp,gleason,pgg45,lpsa
count,9.700000e+01,9.700000e+01,9.700000e+01,9.700000e+01,9.700000e+01,9.700000e+01,9.700000e+01,9.700000e+01,97.000000
mean,-9.614302e-17,-3.216213e-16,3.433679e-16,-4.721309e-17,-1.327689e-16,8.240831e-17,-1.476482e-16,-1.816989e-16,2.478387
std,1.005195e+00,1.005195e+00,1.005195e+00,1.005195e+00,1.005195e+00,1.005195e+00,1.005195e+00,1.005195e+00,1.154329
min,-2.300218e+00,-2.942386e+00,-3.087227e+00,-1.030029e+00,-5.256575e-01,-8.676552e-01,-1.047571e+00,-8.689573e-01,-0.430783
25%,-7.139973e-01,-5.937689e-01,-5.219612e-01,-1.030029e+00,-5.256575e-01,-8.676552e-01,-1.047571e+00,-8.689573e-01,1.731656
50%,8.264956e-02,-1.392703e-02,1.531086e-01,1.383966e-01,-5.256575e-01,-4.450983e-01,3.444069e-01,-3.343557e-01,2.591516
75%,6.626939e-01,5.806076e-01,5.581506e-01,1.010033e+00,-5.256575e-01,9.762744e-01,3.444069e-01,5.566470e-01,3.056357
max,2.107397e+00,2.701661e+00,2.043304e+00,1.542252e+00,1.902379e+00,2.216735e+00,3.128363e+00,2.695054e+00,5.582932


Los promedios de todos los datos normalizados son muy pequeños comparado con los datos, por lo que se puede decir que el promedio es prácticamente cero.

A continuación se realiza una regresión lineal de mínimos cuadrados.

In [17]:
import sklearn.linear_model as lm
X = df_scaled.ix[:,:-1]
N = X.shape[0]
X.insert(X.shape[1], 'intercept', np.ones(N))
y = df_scaled['lpsa']
Xtrain = X[istrain]
ytrain = y[istrain]
Xtest = x[istest]
ytest = y[istest]
linreg = lm.LinearRegression(fit_intercept = False)
linreg.fit(Xtrain, ytrain)

El tercer paso (cuarta línea) agrega una columna llena de números 1, llamada **intercept**, a la derecha de la última columna (*lpsa*). Esta columna representa el valor de $x_0 $ en la matriz de los predictores de tal forma que el intercepto sea $\beta_0$.

Como los datos están normalizados, estos tienen sus promedios centrados y no es necesario calcular el intercepto (es cero)